# Cooperatives Monthly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
from config import Config

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
%sql $Config.SQLALCHEMY_DATABASE_URI

monthly total before running time.

In [ ]:
%%sql firm_registration_filings  <<
SELECT UPPER(a.city),  REPLACE(a.postal_code,' ',''), count(*)
FROM legal_entities le, offices o, addresses a, filings f
WHERE le.id=f.legal_entity_id
AND le.id=o.legal_entity_id
AND o.id=a.office_id
AND a.address_type='mailing'
AND f.filing_type = 'registration'
AND o.office_type = 'businessOffice'
AND to_char(le.founding_date,'yyyymm') = to_char(current_date-27,'yyyymm')
AND le.entity_type in ('SP', 'GP')
GROUP BY upper(a.city),replace(a.postal_code,' ','')
ORDER BY upper(a.city),replace(a.postal_code,' ','');

In [ ]:
last_month = datetime.now() - relativedelta(months=1)
filename = os.path.join(os.getcwd(), r'data/')+'bc_stats_firms_for_' + format(last_month, '%B_%Y') +'.csv'

with open(filename, 'w') as f:
    f.write('FIRM REGISTRATIONS FOR THE MONTH OF ' + format(last_month, '%B_%Y') + ' :\n')
    df=firm_registration_filings.DataFrame()
    if not df.empty:
        df.columns = ['City','Postal_Code','Count']         
    df.to_csv(f, sep=',', encoding='utf-8', index=False)      
    